Import tensorflow and disable v2

In [ ]:
#Importing packages
import time
import tensorflow.compat.v1 as tf
import input_data
import matplotlib.pyplot as plt

tf.disable_v2_behavior()

Load MNIST Data

In [ ]:
#MNIST Dataset has 70000 instances of hand written digits from 0-9, each image size is 28*28
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Define the Hyperparameter & Data info

In [ ]:
#Learning Rate, Iterations, Batch Size Hyperparameters
learning_rate = 0.001
iterations = 10000
batch_size = 128
#dropout = 0.5
#epochs = 10

Create placeholders for Input(X) and Output(Y)

In [ ]:
#Define the placeholders for holding Input and Output Instances
#Here shape is [None, 784], where None means any number of  examples can be passed and 784 stands for 28 * 28
X = tf.placeholder(tf.float32, shape = [None, 784])
#Here shape is [None, 10], where 10 represents number of output classes
Y = tf.placeholder(tf.float32, shape = [None, 10])

Define the weight matrices

In [ ]:
#Defining the weights of 2 hidden layers and output layer
w1 = tf.get_variable(name = 'w1', shape = [784, 512], regularizer = tf.contrib.layers.l2_regularizer(0.001))
w2 = tf.get_variable(name = 'w2', shape = [512, 256], regularizer = tf.contrib.layers.l2_regularizer(0.001))
w3 = tf.get_variable(name = 'w3', shape = [256, 10], regularizer = tf.contrib.layers.l2_regularizer(0.001))

Define the bias 

In [ ]:
#Defining the bias of 2 hidden layers and output layer
b1 = tf.Variable(tf.constant(0, shape = [512], dtype = tf.float32))
b2 = tf.Variable(tf.constant(0, shape = [256], dtype = tf.float32))
b3 = tf.Variable(tf.constant(0, shape = [10], dtype = tf.float32))

Setup the model flow

In [ ]:
#Setting the flow -> Input -> L1 -> A1 -> L2 -> A2 -> Ouput
A1 = tf.nn.relu(tf.matmul(X, w1) + b1)
A2 = tf.nn.relu(tf.matmul(A1, w2) + b2)
#drop = tf.nn.dropout(A3, 0.5)
Yhat = tf.nn.softmax(tf.matmul(A2, w3) + b3)

Define the cost function

In [ ]:
#Cost function definition, using the softmax cross entropy loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Yhat, labels = Y))
final_loss = cross_entropy + tf.reduce_sum(tf.losses.get_regularization_loss())

Metrics to check predictions

In [ ]:
#To verify the predictions
pred = tf.equal(tf.argmax(Yhat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(pred, tf.float32))

Put it all together

In [ ]:
#Setup the optimizer function with learning rate as parameter
#train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(final_loss)

Instantiate the tensorflow global variables -> Graph

In [ ]:
#Initialize the Tensorflow Session and global varibales (Get it ready to spawn a Computational Graph)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Let's train for 10000 iterations

In [ ]:
#Run the for-loop for a well defined number of epochs
s = time.clock()
loss = []
accuracy1 = []
for i in range(iterations + 1):

    #Train in mini-batches
    batchX, batchY = mnist.train.next_batch(batch_size)

    #Generate the Computational Graph and train the network
    sess.run(train_step, feed_dict={X: batchX, Y: batchY})

    #Print step and cost values at end of every 100th step
    if i % 1000 == 0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batchX, Y: batchY})
        
        print('Iteration = {}'.format(i))
        print('Loss = {}'.format(minibatch_loss))
        print('Accuracy = {}\n'.format(minibatch_accuracy))
        
        loss.append(minibatch_loss)
        accuracy1.append(minibatch_accuracy)
    
            
e = time.clock()

print('Time Elapsed = {} \n'.format(e-s))

test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print('Accuracy on Test Set is {}'.format(test_accuracy))

Plotting the graph

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

iterations = np.arange(0, 10001, 1000)
plt.plot(iterations, loss, label = 'Training Loss')
plt.title('Loss graph')
plt.xlabel('Epochs')
plt.xticks(np.arange(0, 10001, 1000))
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
iterations = np.arange(0, 10001, 1000)
plt.plot(iterations, accuracy1, label = 'Training Accuracy')
plt.title('Accuracy graph')
plt.xlabel('Epochs')
plt.xticks(np.arange(0, 10001, 1000))
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Real Time Predictions

In [ ]:
#Import required packages
import numpy as np
from PIL import Image

In [ ]:
#Loading, converting and flattening an image for prediction
img = np.invert(Image.open('test_img.png').convert('L')).ravel()

In [ ]:
#Passing the loaded image to the neural network
prediction = sess.run(tf.argmax(Yhat, 1), feed_dict={X: [img]})
print('Actual Ground Truth is {}'.format(2))
print('Prediction for test image is {}'.format(np.squeeze(prediction)))
plt.imshow(img)

In [ ]:
#Prediction using data from the Test Set randomly (We can select the index randomly)

prediction = sess.run(tf.argmax(Yhat, 1), feed_dict={X: [mnist.test.images[21]]})
print('Actual Ground Truth is {}'.format(np.squeeze(mnist.test.labels[21])))
print('Prediction for test image is {}'.format(np.squeeze(prediction)))
plt.imshow(mnist.test.images[21])